Lab | Random Forests

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [2]:
df1 = pd.read_csv('files_for_lab/categorical.csv')
df2 = pd.read_csv('files_for_lab/numerical.csv')
df3 = pd.read_csv('files_for_lab/target.csv')
data = pd.concat([df1, df2, df3], axis = 1)
data['TARGET_B'].value_counts()

0    90569
1     4843
Name: TARGET_B, dtype: int64

In [3]:
data.shape

(95412, 339)

In [4]:
data.isna().sum()

STATE       0
CLUSTER     0
HOMEOWNR    0
GENDER      0
DATASRCE    0
           ..
HPHONE_D    0
RFA_2F      0
CLUSTER2    0
TARGET_B    0
TARGET_D    0
Length: 339, dtype: int64

In [5]:
data.head()

STATE  CLUSTER HOMEOWNR GENDER  DATASRCE RFA_2R RFA_2A GEOCODE2 DOMAIN_A  \
0    IL       36        H      F         3      L      E        C        T   
1    CA       14        H      M         3      L      G        A        S   
2    NC       43        U      M         3      L      E        C        R   
3    CA       44        U      F         3      L      E        C        R   
4    FL       16        H      F         3      L      F        A        S   

   DOMAIN_B  ODATEW_YR  ODATEW_MM  DOB_YR  DOB_MM  MINRDATE_YR  MINRDATE_MM  \
0         2         89          1      37      12           92            8   
1         1         94          1      52       2           93           10   
2         2         90          1       0       2           91           11   
3         2         87          1      28       1           87           11   
4         2         86          1      20       1           93           10   

   MAXRDATE_YR  MAXRDATE_MM  LASTDATE_YR  LASTDATE_MM  FIRSTDATE_YR  \
0           94            2           95           12            89   
1           95           12           95           12            93   
2           92            7           95           12            90   
3           94           11           95           12            87   
4           96            1           96            1            79   

   FIRSTDATE_MM  TCODE        AGE  INCOME  WEALTH1  HIT  MALEMILI  MALEVET  \
0            11      0  60.000000       5        9    0         0       39   
1            10      1  46.000000       6        9   16         0       15   
2             1      1  61.611649       3        1    2         0       20   
3             2      0  70.000000       1        4    2         0       23   
4             3      0  78.000000       3        2   60         1       28   

   VIETVETS  WWIIVETS  LOCALGOV  STATEGOV  FEDGOV  WEALTH2  POP901  POP902  \
0        34        18        10         2       1        5     992     264   
1        55        11         6         2       1        9    3611     940   
2        29        33         6         8       1        1    7001    2040   
3        14        31         3         0       3        0     640     160   
4         9        53        26         3       2        9    2520     627   

   POP903  POP90C1  POP90C2  POP90C3  POP90C4  POP90C5  ETH1  ETH2  ETH3  \
0     332        0       35       65       47       53    92     1     0   
1     998       99        0        0       50       50    67     0     0   
2    2669        0        2       98       49       51    96     2     0   
3     219        0        8       92       54       46    61     0     0   
4     761       99        0        0       46       54     2    98     0   

   ETH4  ETH5  ETH6  ETH7  ETH8  ETH9  ETH10  ETH11  ETH12  ETH13  ETH14  \
0     0    11     0     0     0     0      0      0      0     11      0   
1    31     6     4     2     6     4     14      0      0      2      0   
2     0     2     0     0     0     0      0      0      0      2      0   
3    11    32     6     2     0     0      0      0      0     31      0   
4     0     1     0     0     0     0      0      0      0      0      0   

   ETH15  ETH16  AGE901  AGE902  AGE903  AGE904  AGE905  AGE906  AGE907  \
0      0      0      39      48      51      40      50      54      25   
1      1      4      34      41      43      32      42      45      32   
2      0      0      35      43      46      37      45      49      23   
3      0      1      32      40      44      34      43      47      25   
4      0      0      33      45      50      36      46      50      27   

   CHIL1  CHIL2  CHIL3  AGEC1  AGEC2  AGEC3  AGEC4  AGEC5  AGEC6  AGEC7  \
0     31     42     27     11     14     18     17     13     11     15   
1     33     46     21     13     14     33     23     10      4      2   
2     35     40     25     13     20     19     16     13     10      8   
3     45     35     20     1

In [7]:
y = data[['TARGET_B','TARGET_D']]
X = data.drop(['TARGET_B', 'TARGET_D'], axis = 1)
X.shape

(95412, 337)

In [8]:
high_cardinality_columns = []
for column in X.columns:
    unique_values = X[column].nunique()
    if unique_values > 14:  # You can adjust the threshold (e.g., 10) based on your preference
        high_cardinality_columns.append((column, unique_values))
if high_cardinality_columns:
    print("Columns with high cardinality:")
    for column, cardinality in high_cardinality_columns:
        print(f"{column}: {cardinality} unique values")
else:
    print("No columns with high cardinality found.")
    
X = X.loc[:, ~X.columns.isin(high_cardinality_columns)]

Columns with high cardinality:
CLUSTER: 53 unique values
ODATEW_YR: 15 unique values
DOB_YR: 96 unique values
MINRDATE_YR: 20 unique values
MAXRDATE_YR: 18 unique values
FIRSTDATE_YR: 26 unique values
TCODE: 55 unique values
AGE: 97 unique values
HIT: 75 unique values
MALEMILI: 95 unique values
MALEVET: 89 unique values
VIETVETS: 95 unique values
WWIIVETS: 100 unique values
LOCALGOV: 55 unique values
STATEGOV: 65 unique values
FEDGOV: 55 unique values
POP901: 9906 unique values
POP902: 4786 unique values
POP903: 5698 unique values
POP90C1: 100 unique values
POP90C2: 100 unique values
POP90C3: 100 unique values
POP90C4: 81 unique values
POP90C5: 81 unique values
ETH1: 100 unique values
ETH2: 100 unique values
ETH3: 85 unique values
ETH4: 96 unique values
ETH5: 100 unique values
ETH6: 21 unique values
ETH7: 61 unique values
ETH8: 60 unique values
ETH9: 61 unique values
ETH10: 33 unique values
ETH11: 34 unique values
ETH12: 43 unique values
ETH13: 98 unique values
ETH14: 45 unique values


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

numerical = X.select_dtypes(np.number)
categorical = X.select_dtypes(include=['object'])
display(categorical.head())

for col in categorical.columns:
    print(col, categorical[col].nunique())
print()

encoder = OneHotEncoder(drop='first').fit(categorical)
encoded_categorical = encoder.transform(categorical).toarray()
encoded_categorical = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out(), index=categorical.index)

# Reset the index of numericalX and encoded_categorical
#numericalX1 = numericalX1.reset_index(drop=True)
#encoded_categorical = encoded_categorical.reset_index(drop=True)

# Concatenate numericalX and encoded_categorical along axis=1
X = pd.concat([numerical, encoded_categorical], axis=1)

# Note: We need to do train/test split before downsampling, and then only downsample the training set - Why?
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

STATE HOMEOWNR GENDER RFA_2R RFA_2A GEOCODE2 DOMAIN_A
0    IL        H      F      L      E        C        T
1    CA        H      M      L      G        A        S
2    NC        U      M      L      E        C        R
3    CA        U      F      L      E        C        R
4    FL        H      F      L      F        A        S

STATE 12
HOMEOWNR 2
GENDER 3
RFA_2R 1
RFA_2A 4
GEOCODE2 4
DOMAIN_A 5



In [11]:
data['TARGET_B'].value_counts()

0    90569
1     4843
Name: TARGET_B, dtype: int64

In [12]:
# for upsampling we need to temporarily concat X_train and y_train
trainset = pd.concat([X_train, y_train], axis=1)

In [13]:
# Let's upsample the minority 
from sklearn.utils import resample
df_majority = trainset[trainset['TARGET_B'] == 0]
df_minority = trainset[trainset['TARGET_B'] == 1]

df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=len(df_majority),    # to match majority class
                                 random_state=123) # reproducible results
trainset1 = pd.concat([df_majority, df_minority_upsampled], axis=0)
trainset1

CLUSTER  DATASRCE  DOMAIN_B  ODATEW_YR  ODATEW_MM  DOB_YR  DOB_MM  \
79401       24         3         1         86          1      23       1   
86429       53         2         3         96          1      45       1   
38838        5         3         2         89          1      21       1   
83012       27         3         2         94          1      43       1   
33395       36         2         2         88          1      16       7   
...        ...       ...       ...        ...        ...     ...     ...   
80175       29         1         2         90          1      24       9   
5000        17         3         2         89          1      18       2   
29030       16         3         2         94          1      38       1   
9812        30         1         3         92          1      48       4   
31094       45         3         2         86          1       0       2   

       MINRDATE_YR  MINRDATE_MM  MAXRDATE_YR  MAXRDATE_MM  LASTDATE_YR  \
79401           89            2           90           12           96   
86429           96            2           96            2           96   
38838           88           11           93            9           95   
83012           94            1           95            4           95   
33395           89           12           89            9           95   
...            ...          ...          ...          ...          ...   
80175           95            3           95            1           95   
5000            89            2           95           12           95   
29030           96            2           95            1           96   
9812            92           11           93           12           95   
31094           90            1           92            5           95   

       LASTDATE_MM  FIRSTDATE_YR  FIRSTDATE_MM  TCODE        AGE  INCOME  \
79401            1            88             3      1  75.000000       4   
86429            2            96             2      0  53.000000       5   
38838           12            88            11      1  77.000000       6   
83012            4            94             1      3  55.000000       5   
33395           10            88            12      0  81.000000       4   
...            ...           ...           ...    ...        ...     ...   
80175           10            90            10      1  73.000000       3   
5000            12            88             9      0  80.000000       2   
29030            2            94             7     28  60.000000       5   
9812             8            92             2      0  50.000000       1   
31094            7            88             2      1  61.611649       6   

       WEALTH1  HIT  MALEMILI  MALEVET  VIETVETS  WWIIVETS  LOCALGOV  \
79401        6    2         0       31        10        68         6   
86429        9    0         0       29        24        38         7   
38838        6    9         1       40        23        41         8   
83012        2   21        33       27        29        18        12   
33395        9    0         0       48        11        52         0   
...        ...  ...       ...      ...       ...       ...       ...   
80175        5   20         0       51        15        70        12   
5000         4   25         2       29        27        19         5   
29030        9    0         0       32        47        41         7   
9812         1    4         0       48        22        24         2   
31094        1   12         1       30        23        36         9   

       STATEGOV  FEDGOV  WEALTH2  POP901  POP902  POP903  POP90C1  POP90C2  \
79401        12       3        9     706     203     272       99        0   
86429         8       4        9    4526    1232    1592        0        0   
38838         3       4        6    1388     397     486       99        0   
83012         2       7        9     991     268     389       93        0   
33395         6       6        6     

In [14]:
X_train = trainset1.drop(['TARGET_B', 'TARGET_D'], axis=1)
y_train = trainset1[['TARGET_B', 'TARGET_D']]

In [15]:
X_train.head()

CLUSTER  DATASRCE  DOMAIN_B  ODATEW_YR  ODATEW_MM  DOB_YR  DOB_MM  \
79401       24         3         1         86          1      23       1   
86429       53         2         3         96          1      45       1   
38838        5         3         2         89          1      21       1   
83012       27         3         2         94          1      43       1   
33395       36         2         2         88          1      16       7   

       MINRDATE_YR  MINRDATE_MM  MAXRDATE_YR  MAXRDATE_MM  LASTDATE_YR  \
79401           89            2           90           12           96   
86429           96            2           96            2           96   
38838           88           11           93            9           95   
83012           94            1           95            4           95   
33395           89           12           89            9           95   

       LASTDATE_MM  FIRSTDATE_YR  FIRSTDATE_MM  TCODE   AGE  INCOME  WEALTH1  \
79401            1            88             3      1  75.0       4        6   
86429            2            96             2      0  53.0       5        9   
38838           12            88            11      1  77.0       6        6   
83012            4            94             1      3  55.0       5        2   
33395           10            88            12      0  81.0       4        9   

       HIT  MALEMILI  MALEVET  VIETVETS  WWIIVETS  LOCALGOV  STATEGOV  FEDGOV  \
79401    2         0       31        10        68         6        12       3   
86429    0         0       29        24        38         7         8       4   
38838    9         1       40        23        41         8         3       4   
83012   21        33       27        29        18        12         2       7   
33395    0         0       48        11        52         0         6       6   

       WEALTH2  POP901  POP902  POP903  POP90C1  POP90C2  POP90C3  POP90C4  \
79401        9     706     203     272       99        0        0       47   
86429        9    4526    1232    1592        0        0       99       49   
38838        6    1388     397     486       99        0        0       49   
83012        9     991     268     389       93        0        7       50   
33395        6     529     153     225        0       99        0       46   

       POP90C5  ETH1  ETH2  ETH3  ETH4  ETH5  ETH6  ETH7  ETH8  ETH9  ETH10  \
79401       53    75    21     0     2     2     0     0     0     0      0   
86429       51    75    16     9     0     1     0     0     0     0      0   
38838       51    78     1     1    15    11     0     1     5     6      1   
83012       50    88     8     0     3     4     0     0     0     1      1   
33395       54    99     0     0     1     0     0     0     0     0      0   

       ETH11  ETH12  ETH13  ETH14  ETH15  ETH16  AGE901  AGE902  AGE903  \
79401      1      0      0      1      0      2      34      42      46   
86429      0      0      1      0      0      0      35      47      51   
38838      1      1      7      1      0      3      40      49      53   
83012      0      0      2      1      0      1      26      32      35   
33395      0      0      0      0      0      0      39      48      55   

       AGE904  AGE905  AGE906  AGE907  CHIL1  CHIL2  CHIL3  AGEC1  AGEC2  \
79401      36      45      49      25     36     47     17     13     21   
86429      38      49      53      28     39     40     20     11     18   
38838      40      48      52      21     34     44     23     10     16   
83012      27      36      40      30     54     34     12     20     38   
33395      42      51      55      21     34     44     22     10     18   

       AGEC3  AGEC4  AGEC5  AGEC6  AGEC7  CHILC1  CHILC2  CHILC3  CHILC4  \
79401     19     13     15     12      6      14      14      35      21   
86429     16     15     14     12     14      14      18      34      19   
38838     17     14     22     14      6      15      13 

In [16]:
y_train.head()

TARGET_B  TARGET_D
79401         0       0.0
86429         0       0.0
38838         0       0.0
83012         0       0.0
33395         0       0.0

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

clf = RandomForestClassifier(max_depth=5,
                             min_samples_split=20,
                             min_samples_leaf =20,
                             max_samples=0.8,
                             random_state = 42)

clf.fit(X_train, y_train['TARGET_B'])

print("The Accuracy for the Random Forest in the TRAIN set is {:.2f}".format(clf.score(X_train, y_train['TARGET_B'])))
print("The Accuracy for the Random Forest in the TEST  set is {:.2f}".format(clf.score(X_test, y_test['TARGET_B'])))

y_pred = clf.predict(X_test)
display(y_test['TARGET_B'].value_counts())
display(confusion_matrix(y_test['TARGET_B'], y_pred))

The Accuracy for the Random Forest in the TRAIN set is 0.62
The Accuracy for the Random Forest in the TEST  set is 0.61


0    18083
1     1000
Name: TARGET_B, dtype: int64

array([[11026,  7057],
       [  452,   548]])

In [18]:
# y=0, y_pred=0, earnings = 0
# y=0, y_pred=1, earnings = 0
# y=1, y_pred=0, earnings = -target_d
# y=1, y_pred=1, earnings = target_d

In [20]:
from sklearn.metrics import make_scorer, recall_score
from sklearn.model_selection import GridSearchCV

recall_scorer = make_scorer(recall_score)

param_grid = {
    'max_depth': [5, 7], # 3 * 2 * 2 * 2 * 1 = 3 * 8 = 24
    'min_samples_split': [5, 10],
    'min_samples_leaf' : [5, 10],
    'max_samples':[0.6, 0.8]
    }

rf_classifier = RandomForestClassifier(random_state=100)

grid_search = GridSearchCV(rf_classifier, param_grid, cv=5, return_train_score=True, n_jobs=-1, verbose = 20, scoring=recall_scorer)
grid_search.fit(X_train, y_train['TARGET_B'])
grid_search.best_params_ #To check the best set of parameters returned


Fitting 5 folds for each of 16 candidates, totalling 80 fits


{'max_depth': 7,
 'max_samples': 0.6,
 'min_samples_leaf': 5,
 'min_samples_split': 5}